In [ ]:
############# TRATATIVA E NORMALIZAÇÃO DOS DADOS ######################
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV 
import joblib 
import os 

df_obesity = pd.read_csv("C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\data\\Obesity.csv")


# Passos de exploração baixo estão descrito no Exploracao.ipynb
novos_nomes = {
    'Gender': 'Gênero',
    'Age': 'Idade',
    'Height': 'Altura',
    'Weight': 'Peso',
    'family_history': 'Histórico_Familiar_Obesidade',
    'FAVC': 'Frequencia_Consumo_Alimento_Calorico',
    'FCVC': 'Frequencia_Consumo_Vegetais',
    'NCP': 'Numero_Refeicoes_Principais',
    'CAEC': 'Consumo_Alimento_Entre_Refeicoes',
    'SMOKE': 'Fumante',
    'CH2O': 'Consumo_Agua',
    'SCC': 'Monitoramento_Calorico',
    'FAF': 'Frequencia_Atividade_Fisica',
    'TUE': 'Tempo_Uso_Tecnologia',
    'CALC': 'Consumo_Alcool',
    'MTRANS': 'Meio_Transporte',
    'Obesity': 'Status_Obesidade'  
}

df_obesity = df_obesity.rename(columns=novos_nomes)

#Colunas de sim e nao para 1 e 0
colunas_sim_nao = ['Histórico_Familiar_Obesidade', 'Frequencia_Consumo_Alimento_Calorico', 'Fumante', 'Monitoramento_Calorico']

dict_sin_nao = {'yes': 1, 'no': 0}

df_obesity[colunas_sim_nao] = df_obesity[colunas_sim_nao].replace(dict_sin_nao).astype(int)


#coluna gênero para 0 e 1

coluna_genero = ['Gênero']
dic_genero = {'Female' : 0,'Male': 1}

df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_genero).astype(int)



#colunas com classificacao de 0 a 3 para valores numericos
colunas_class_0_3 = ['Consumo_Alcool','Consumo_Alimento_Entre_Refeicoes']
dic_class_0_3 = {'no': 0,'Sometimes': 1,'Frequently': 2, 'Always': 3}

df_obesity[colunas_class_0_3] = df_obesity[colunas_class_0_3].replace(dic_class_0_3).astype(int)


#colunas com classificacao de trasportes dividida em 3 
# 0 para baixa intensidade 
# 1 para meia intensidade
# 2 para alta intensidade

colunas_trasporte = ['Meio_Transporte']
dic_trasporte = {'Walking': 2,'Public_Transportation': 1,'Automobile': 0,
                 'Motorbike': 0,'Bike': 2}

df_obesity[colunas_trasporte] = df_obesity[colunas_trasporte].replace(dic_trasporte).astype(int)



### Colunas onde ponto flutuante está incorreto, pegando o primeiro digito

colunas_primeiro_digito = [ 'Tempo_Uso_Tecnologia','Frequencia_Atividade_Fisica',
'Consumo_Agua','Numero_Refeicoes_Principais','Frequencia_Consumo_Vegetais']

for coluna in colunas_primeiro_digito:
    df_obesity[coluna] = df_obesity[coluna].astype(str).str[0]

    df_obesity[coluna] = df_obesity[coluna].astype(int)

# Coluna de idade arredondado
df_obesity['Idade'] = df_obesity['Idade'].astype(int)

#Colunas de peso e altura 2 casa decimais
df_obesity['Peso'] = df_obesity['Peso'].astype(float).round(2)
df_obesity['Altura'] = df_obesity['Altura'].astype(float).round(2)


#IREMOS TESTAR 3 ABORDAGENS 

df_obesity_3_class = df_obesity.copy()
df_obesity_biaria = df_obesity.copy()
df_obesity_4_class = df_obesity.copy()


C:\Users\Igor\AppData\Local\Temp\ipykernel_16132\2651140745.py:49: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_obesity[colunas_sim_nao] = df_obesity[colunas_sim_nao].replace(dict_sin_nao).astype(int)
C:\Users\Igor\AppData\Local\Temp\ipykernel_16132\2651140745.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_obesity['Gênero'] = df_obesity['Gênero'].replace(dic_genero).astype(int)
C:\Users\Igor\AppData\Local\Temp\ipykernel_16132\2651140745.py:65: FutureWarning: Downcasting behavior in `replace` is deprecated and will be 

In [ ]:

# 1- CLASSIFICANDO OBESIDADE EM 2 NIVEIS: OBESO E NAO OBESO

#Coluna de classificacao de obesiade para valor numerico



def normalize_obesity_binario(binario):
    if binario in ['Insufficient_Weight', 'Normal_Weight', 'Overweight_Level_I', 'Overweight_Level_II']:
        return 0  # Não obeso
    else:
        return 1  # Obeso

df_obesity_biaria["Status_Obesidade"] = df_obesity_biaria["Status_Obesidade"].apply(normalize_obesity_binario)


#separando as caracteriscas do target
X = df_obesity_biaria.drop(['Status_Obesidade','Altura','Peso'], axis=1) # CARACTERISTICAS
y = df_obesity_biaria['Status_Obesidade'] # TARGETS

#separando a base de teste e treino
X_treino_binario, X_teste_binario, y_treino_binario, y_teste_binario = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


##############Treinando os mode #####################

#Regressao LogisticRegression
log_reg_model_binario = LogisticRegression(random_state=42) 

log_reg_model_binario.fit(X_treino_binario, y_treino_binario)

#Random Forest
rf_model_binario = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_binario.fit(X_treino_binario, y_treino_binario)

#XGBoost
xgb_model_binario = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_binario.fit(X_treino_binario, y_treino_binario)


# Usando os dados de testes
y_pred_log_reg_binario = log_reg_model_binario.predict(X_teste_binario)
y_pred_rf_binario = rf_model_binario.predict(X_teste_binario)
y_pred_xgb_binario = xgb_model_binario.predict(X_teste_binario)


print("--- Regressão Logística ---") 
print(classification_report(y_teste_binario, y_pred_log_reg_binario))

print("--- Random Forest ---") 
print(classification_report(y_teste_binario, y_pred_rf_binario))

print("--- XGBoost ---") 
print(classification_report(y_teste_binario, y_pred_xgb_binario))

c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\xgboost\training.py:199: UserWarning: [19:05:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--- Regressão Logística ---
              precision    recall  f1-score   support

           0       0.79      0.66      0.72       228
           1       0.67      0.80      0.73       195

    accuracy                           0.73       423
   macro avg       0.73      0.73      0.73       423
weighted avg       0.74      0.73      0.73       423

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.90      0.91      0.90       228
           1       0.89      0.88      0.89       195

    accuracy                           0.90       423
   macro avg       0.90      0.89      0.90       423
weighted avg       0.90      0.90      0.90       423

--- XGBoost ---
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       228
           1       0.91      0.90      0.90       195

    accuracy                           0.91       423
   macro avg       0.91      0.91      0.91       423
weighted

In [ ]:

# 2- CLASSIFICANDO OBESIDADE EM 3 NIVEIS: NORMAL, SOBREPESO E OBESO

#Coluna de classificacao de obesiade para valor numerico


def normalize_obesity_3(level):
    if level in ['Insufficient_Weight', 'Normal_Weight']:
        return 0  # Peso normal
    elif level in ['Overweight_Level_I', 'Overweight_Level_II']:
        return 1  # Sobrepeso
    else:
        return 2  # Obeso



df_obesity_3_class["Status_Obesidade"] = df_obesity_3_class["Status_Obesidade"].apply(normalize_obesity_3)



#separando as caracteriscas do target
X = df_obesity_3_class.drop(['Status_Obesidade','Altura','Peso'], axis=1) # CARACTERISTICAS
y = df_obesity_3_class['Status_Obesidade'] # TARGETS


#separando a base de teste e treino
X_treino_3_class, X_teste_3_class, y_treino_3_class, y_teste_3_class = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)



#############Treinando os mode #####################

#Regressao LogisticRegression
log_reg_model_3_class = LogisticRegression(random_state=42) 
log_reg_model_3_class.fit(X_treino_3_class, y_treino_3_class)

#Random Forest
rf_model_3_class = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_3_class.fit(X_treino_3_class, y_treino_3_class)

#XGBoost
xgb_model_3_class = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_3_class.fit(X_treino_3_class, y_treino_3_class)



# Usando os dados de testes
y_pred_log_reg_3_class = log_reg_model_3_class.predict(X_teste_3_class)
y_pred_rf_3_class = rf_model_3_class.predict(X_teste_3_class)
y_pred_xgb_3_class = xgb_model_3_class.predict(X_teste_3_class)


print("--- Regressão Logística ---") 
print(classification_report(y_teste_3_class, y_pred_log_reg_3_class))

print("--- Random Forest ---") 
print(classification_report(y_teste_3_class, y_pred_rf_3_class))

print("--- XGBoost ---") 
print(classification_report(y_teste_3_class, y_pred_xgb_3_class))

c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\xgboost\training.py:199: UserWarning: [19:05:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--- Regressão Logística ---
              precision    recall  f1-score   support

           0       0.81      0.79      0.80       112
           1       0.55      0.29      0.38       116
           2       0.70      0.91      0.79       195

    accuracy                           0.71       423
   macro avg       0.69      0.66      0.66       423
weighted avg       0.69      0.71      0.68       423

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.87      0.90      0.89       112
           1       0.79      0.82      0.80       116
           2       0.93      0.89      0.91       195

    accuracy                           0.87       423
   macro avg       0.86      0.87      0.87       423
weighted avg       0.87      0.87      0.87       423

--- XGBoost ---
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       112
           1       0.79      0.84      0.82       116
         

In [ ]:

# 3- CLASSIFICANDO OBESIDADE EM 4 NIVEIS: ABAIXO DO PESO, NORMAL, SOBREPESO E OBESO

#Coluna de classificacao de obesiade para valor numerico

df_obesity_4_class = df_obesity

def normalize_obesity_4(level):
    if level in ['Insufficient_Weight']:
        return 0  # Abaxio do peso
    elif level in ['Normal_Weight']:
        return 1  # Peso normal
    elif level in ['Overweight_Level_I', 'Overweight_Level_II']:
        return 2  # Sobrepeso
    else:
        return 3  # Obeso

df_obesity_4_class["Status_Obesidade"] = df_obesity_4_class["Status_Obesidade"].apply(normalize_obesity_4)


#separando as caracteriscas do target
X = df_obesity_4_class.drop(['Status_Obesidade','Altura','Peso'], axis=1) # CARACTERISTICAS
y = df_obesity_4_class['Status_Obesidade'] # TARGETS

#separando a base de teste e treino
X_treino_4_class, X_teste_4_class, y_treino_4_class, y_teste_4_class = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)


##############Treinando os mode #####################

#Regressao LogisticRegression
log_reg_model_4_class = LogisticRegression(random_state=42) 
log_reg_model_4_class.fit(X_treino_4_class, y_treino_4_class)

#Random Forest
rf_model_4_class = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model_4_class.fit(X_treino_4_class, y_treino_4_class)

#XGBoost
xgb_model_4_class = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model_4_class.fit(X_treino_4_class, y_treino_4_class)


# Usando os dados de testes
y_pred_log_reg_4_class = log_reg_model_4_class.predict(X_teste_4_class)
y_pred_rf_4_class = rf_model_4_class.predict(X_teste_4_class)
y_pred_xgb_4_class = xgb_model_4_class.predict(X_teste_4_class)


print("--- Regressão Logística ---") 
print(classification_report(y_teste_4_class, y_pred_log_reg_4_class))

print("--- Random Forest ---") 
print(classification_report(y_teste_4_class, y_pred_rf_4_class))

print("--- XGBoost ---") 
print(classification_report(y_teste_4_class, y_pred_xgb_4_class))

c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Igor\Documents\GitHub\Tech_Challenge_Fase_4_Data_Analitycs\venv_tc4\Lib\site-packages\xgboost\training.py:199: UserWarning: [19:05:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--- Regressão Logística ---
              precision    recall  f1-score   support

           0       0.61      0.65      0.63        54
           1       0.47      0.34      0.40        58
           2       0.61      0.26      0.36       116
           3       0.65      0.91      0.75       195

    accuracy                           0.62       423
   macro avg       0.58      0.54      0.54       423
weighted avg       0.61      0.62      0.58       423

--- Random Forest ---
              precision    recall  f1-score   support

           0       0.81      0.89      0.85        54
           1       0.60      0.57      0.58        58
           2       0.76      0.78      0.77       116
           3       0.91      0.89      0.90       195

    accuracy                           0.81       423
   macro avg       0.77      0.78      0.77       423
weighted avg       0.81      0.81      0.81       423

--- XGBoost ---
              precision    recall  f1-score   support

         

In [ ]:
# 1 Definindo os dados de teste (as 4 pessoas)
data_test = {
    'Gênero': [0, 1, 0, 1], # 0=F, 1=M
    'Idade': [25, 35, 45, 55],
    'Histórico_Familiar_Obesidade': [0, 1, 1, 1],# 0=No, 1=Yes
    'Frequencia_Consumo_Alimento_Calorico': [0, 1, 1, 1],# 0=No, 1=Yes
    'Frequencia_Consumo_Vegetais': [3, 2, 1, 2], # 0 a 3
    'Numero_Refeicoes_Principais': [3, 3, 2, 3], # 0 a 3
    'Consumo_Alimento_Entre_Refeicoes': [1, 2, 3, 1], # 0=no, 1=Sometimes, 2=Frequently, 3=Always
    'Fumante': [0, 0, 1, 0], # 0=No, 1=Yes
    'Consumo_Agua': [3, 2, 1, 2], # # 0 a 3
    'Monitoramento_Calorico': [0, 0, 0, 0], # 0=No, 1=Yes
    'Frequencia_Atividade_Fisica': [2, 1, 0, 1], # # 0 a 2
    'Tempo_Uso_Tecnologia': [1, 2, 3, 0], #  0 a 3
    'Consumo_Alcool': [1, 1, 2, 0], # 0=no, 1=Sometimes, 2=Frequently, 3=Always
    'Meio_Transporte': [2, 1, 0, 0] # 0=Baixa (Auto/Moto), 1=Média (Public), 2=Alta (Walk/Bike)
}

df_teste = pd.DataFrame(data_test)

# garantindo que a odem seja a emsma do treino
colunas_treinamento = X_treino_binario.columns 
df_teste = df_teste[colunas_treinamento]


# Função auxiliar para mapear os códigos numéricos de volta para as classes de obesidade
def map_results(predictions, classes_type):
    if classes_type == 'binaria':
        mapping = {0: 'Não Obeso', 1: 'Obeso'}
    elif classes_type == '3_classes':
        mapping = {0: 'Normal', 1: 'Sobrepeso', 2: 'Obeso'}
    elif classes_type == '4_classes':
        mapping = {0: 'Abaixo do Peso', 1: 'Peso Normal', 2: 'Sobrepeso', 3: 'Obeso'}
    else:
        return predictions
    return [mapping.get(p, f'Classe {p}') for p in predictions]


# 2 Fazendo as previsões

# --- Abordagem 1: binaria (0=Não Obeso, 1=Obeso) 
y_pred_log_reg_binario = log_reg_model_binario.predict(df_teste)
y_pred_rf_binario = rf_model_binario.predict(df_teste)
y_pred_xgb_binario = xgb_model_binario.predict(df_teste)

# --- Abordagem 2: 3 Classes (0=Normal, 1=Sobrepeso, 2=Obeso) 
y_pred_log_reg_3_class = log_reg_model_3_class.predict(df_teste)
y_pred_rf_3_class = rf_model_3_class.predict(df_teste)
y_pred_xgb_3_class = xgb_model_3_class.predict(df_teste)

# --- Abordagem 3: 4 Classes (0=Abaixo do Peso, 1=Normal, 2=Sobrepeso, 3=Obeso)
y_pred_log_reg_4_class = log_reg_model_4_class.predict(df_teste)
y_pred_rf_4_class = rf_model_4_class.predict(df_teste)
y_pred_xgb_4_class = xgb_model_4_class.predict(df_teste)

# 3. Organizando os resultados

resultados_binaria = pd.DataFrame({
    'Pessoa': [f'Pessoa {i+1}' for i in range(4)],
    'Regressão Logística': map_results(y_pred_log_reg_binario, 'binaria'),
    'Random Forest': map_results(y_pred_rf_binario, 'binaria'),
    'XGBoost': map_results(y_pred_xgb_binario, 'binaria')
})

resultados_3_classes = pd.DataFrame({
    'Pessoa': [f'Pessoa {i+1}' for i in range(4)],
    'Regressão Logística': map_results(y_pred_log_reg_3_class, '3_classes'),
    'Random Forest': map_results(y_pred_rf_3_class, '3_classes'),
    'XGBoost': map_results(y_pred_xgb_3_class, '3_classes')
})

resultados_4_classes = pd.DataFrame({
    'Pessoa': [f'Pessoa {i+1}' for i in range(4)],
    'Regressão Logística': map_results(y_pred_log_reg_4_class, '4_classes'),
    'Random Forest': map_results(y_pred_rf_4_class, '4_classes'),
    'XGBoost': map_results(y_pred_xgb_4_class, '4_classes')
})

print("Resultados da Classificação Binária (Não Obeso / Obeso):")
print(resultados_binaria.to_markdown(index=False))
print("\n" + "-"*50 + "\n")

print("Resultados da Classificação em 3 Classes (Normal / Sobrepeso / Obeso):")
print(resultados_3_classes.to_markdown(index=False))
print("\n" + "-"*50 + "\n")

print("Resultados da Classificação em 4 Classes (Abaixo do Peso / Normal / Sobrepeso / Obeso):")
print(resultados_4_classes.to_markdown(index=False))

Resultados da Classificação Binária (Não Obeso / Obeso):
| Pessoa   | Regressão Logística   | Random Forest   | XGBoost   |
|:---------|:----------------------|:----------------|:----------|
| Pessoa 1 | Não Obeso             | Não Obeso       | Não Obeso |
| Pessoa 2 | Não Obeso             | Não Obeso       | Não Obeso |
| Pessoa 3 | Não Obeso             | Não Obeso       | Obeso     |
| Pessoa 4 | Obeso                 | Não Obeso       | Não Obeso |

--------------------------------------------------

Resultados da Classificação em 3 Classes (Normal / Sobrepeso / Obeso):
| Pessoa   | Regressão Logística   | Random Forest   | XGBoost   |
|:---------|:----------------------|:----------------|:----------|
| Pessoa 1 | Normal                | Normal          | Normal    |
| Pessoa 2 | Sobrepeso             | Sobrepeso       | Sobrepeso |
| Pessoa 3 | Sobrepeso             | Sobrepeso       | Obeso     |
| Pessoa 4 | Obeso                 | Sobrepeso       | Sobrepeso |

--------------

In [ ]:
########## Selecionando melhor modelo ############

# Seu modelo treinado é `xgb_model_3_class` com 88 de acuracia
joblib.dump(xgb_model_3_class, f'C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\src\\modelos\\modelo_obesidade_xgb_model_3_class.pkl')

['C:\\Users\\Igor\\Documents\\GitHub\\Tech_Challenge_Fase_4_Data_Analitycs\\src\\modelo_obesidade_xgb_model_3_class.pkl']